In [71]:
import pandas as pd
from pathlib import Path
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import VarianceThreshold, mutual_info_classif, SelectKBest, chi2, SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from skfeature.function.similarity_based import fisher_score


In [72]:
starter_df = pd.read_csv(Path("c://users/ajcth/documents/github/bank_churn_model/Resources/BankChurnersPrimary.csv"))
starter_df.sample(5)

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
6252,712222758,Attrited Customer,38,F,3,Graduate,Married,Less than $40K,Blue,36,...,2300.0,0,2300.0,0.703,2500,47,0.119,0.000,0.994990,0.00501
3629,713004933,Attrited Customer,40,M,3,High School,Married,$60K - $80K,Blue,27,...,6813.0,2332,4481.0,0.580,2634,51,0.378,0.342,0.985760,0.01424
7864,715516458,Attrited Customer,42,F,1,High School,Married,$40K - $60K,Blue,36,...,2705.0,2430,275.0,0.698,2602,43,0.433,0.898,0.993950,0.00605
5871,717123858,Existing Customer,36,F,1,Graduate,Married,$40K - $60K,Blue,36,...,3635.0,1617,2018.0,0.828,4045,62,1.067,0.445,0.000087,0.99991
8877,717471933,Existing Customer,45,M,2,Unknown,Married,$40K - $60K,Blue,27,...,5800.0,1434,4366.0,0.715,8202,89,0.679,0.247,0.000210,0.99979


In [73]:
starter_df.drop(columns=[
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2', 
'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
'CLIENTNUM',
'Education_Level', 
'Marital_Status', 
'Income_Category'
], 
inplace= True)

In [74]:

enc = OneHotEncoder(sparse=False)
categorical_variables = ['Card_Category']
encoded_data = enc.fit_transform(starter_df[categorical_variables])
encoded_dataframe = pd.DataFrame(encoded_data, columns = enc.get_feature_names(categorical_variables))
starter_df.drop(columns=['Card_Category'], inplace=True)
starter_df = pd.concat([starter_df, encoded_dataframe.set_axis(starter_df.index)], axis=1)
starter_df.sample(10)

c:\python_files\Anaconda3\envs\dev\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Attrition_Flag,Customer_Age,Gender,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Card_Category_Blue,Card_Category_Gold,Card_Category_Platinum,Card_Category_Silver
7024,Existing Customer,35,F,3,28,5,6,2,3250.0,1560,1690.0,0.634,4673,73,0.698,0.480,1.0,0.0,0.0,0.0
840,Existing Customer,64,F,0,51,6,3,1,4920.0,1142,3778.0,0.507,1420,31,0.348,0.232,1.0,0.0,0.0,0.0
7513,Existing Customer,46,F,2,31,5,2,3,8243.0,927,7316.0,0.568,4503,90,0.765,0.112,1.0,0.0,0.0,0.0
803,Existing Customer,30,M,1,17,3,3,2,7906.0,1631,6275.0,0.889,1946,45,0.552,0.206,1.0,0.0,0.0,0.0
375,Existing Customer,49,F,4,44,6,3,2,18206.0,1586,16620.0,0.921,1439,48,1.000,0.087,1.0,0.0,0.0,0.0
5162,Existing Customer,54,F,4,48,3,2,2,9723.0,1307,8416.0,0.803,4043,76,0.900,0.134,1.0,0.0,0.0,0.0
9806,Existing Customer,31,M,1,23,5,1,2,13427.0,1562,11865.0,0.537,13117,108,0.714,0.116,1.0,0.0,0.0,0.0
6964,Existing Customer,59,F,1,48,4,2,3,5528.0,1823,3705.0,0.731,4857,81,0.723,0.330,1.0,0.0,0.0,0.0
4564,Existing Customer,58,F,1,46,4,1,3,2799.0,1532,1267.0,0.583,4214,83,0.537,0.547,1.0,0.0,0.0,0.0
2782,Existing Customer,51,F,0,35,5,3,4,3132.0,1815,1317.0,1.018,3607,60,0.935,0.580,1.0,0.0,0.0,0.0


In [75]:
starter_df = starter_df.replace({'Gender' : {
'M': 0, 
'F': 1
}})
starter_df.Gender = starter_df.Gender.astype(int)

In [76]:
starter_df = starter_df.replace({'Attrition_Flag' : {
'Attrited Customer': 0, 
'Existing Customer': 1
}})
starter_df.Attrition_Flag = starter_df.Attrition_Flag.astype(int)

In [77]:
y = starter_df['Attrition_Flag']
X = starter_df.drop(columns=['Attrition_Flag'])

In [78]:
gbc = GradientBoostingClassifier()
sfs = SequentialFeatureSelector(gbc, n_features_to_select=6, direction='forward')
sfs.fit(X, y)
X_selection = sfs.transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_selection, y, random_state=1)

In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [ ]:
gbc_model = GradientBoostingClassifier(
n_estimators=500,
random_state= 2
)

In [ ]:
gbc_model.fit(X_train, y_train)

In [ ]:
gbc_test_predictions = gbc_model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, gbc_test_predictions)
print("Accuracy: {:.2f}%".format(accuracy * 100))

In [ ]:
gbc_test_matrix = confusion_matrix(y_test, gbc_test_predictions)
print(gbc_test_matrix)

In [ ]:
gbc_testing_report = classification_report(y_test, gbc_test_predictions)
print(gbc_testing_report)

In [ ]:
imb_gbc_testing_report = classification_report_imbalanced(y_test, gbc_test_predictions)
print(imb_gbc_testing_report)

In [ ]:
sfs.get_support().tolist()

### If 8 features were picked:
Customer_Age, Total_Relationship_Count, Months_Inactive_12_mon, Total_Revolving_Bal, Total_Amt_Chng_Q4_Q1, Total_Trans_Amt, Total_Trans_Ct, Total_Ct_Chng_Q4_Q1
